In [61]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 11 16:59:35 2020
Math Methods Coursework 3 skeleton code

@author: sschultz
"""

import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy.signal import argrelextrema
from matplotlib.colors import ListedColormap

# %% Load dataset

datadir = 'data/dataset3_m62/'
trackdata = io.loadmat(datadir+'downTrackdata.mat')
phi = trackdata['phi']; x = trackdata['x']; y = trackdata['y']; r=trackdata['r'];
speed = trackdata['speed']; t = trackdata['time'];
spikedata = io.loadmat(datadir+'spikes.mat')
spikes = spikedata['spikes']
params = spikedata['params']
fluodata = io.loadmat(datadir+'fissa_output.mat')
dFonF = fluodata['ddf_f'] # fluorescence time-series variable
C = np.size(dFonF,0) # number of cells
fs = 30.92 # sampling rate in Hz - everything is resampled at this rate
L = np.size(dFonF,1)
t = np.linspace(0,L*1/fs,L) # duration of recording in sec

# %% Useful functions provided

# function to break up the fluorescence response of each cell into laps
# takes as input:
# sp: a vector of a cell's activity binarized into 0,1 (spiking/not-spiking) 
# phi_bin: sequence of angular position values, binned into nphi bins
# nlaps: 
def get_lapresponse(sp,phi_bin,nlaps,nphi):
    response = np.zeros((nlaps,nphi))
    lap=0
    phi0=phi_bin[0]
    lapflag=True  # flag to ensure we only increment lap counter on first time in bin
    for i in range(0,L):
        if  sp[i]:
            response[lap][phi_bin[i]-1] += 1
        # and augment lap counter
        if (phi_bin[i]==phi0 and lapflag==True):
            lap += 1
            lapflag=False # put down flag until we are past this bin
            #print(lap)
        if phi_bin[i]>phi0:
            lapflag=True

    return r

# %% some basic processing you will need
# bin the spatial variable phi
nbins_phi = 20
edges = np.arange(0,360,360/nbins_phi)
phi_bin = np.digitize(phi,edges)

# then how many laps does the mouse do?
# use big jumps in position to mark clocking around circular path
dphi = np.diff(phi_bin,axis=0)
#plt.plot(t[:-1],dphi,'r-')
nlaps = np.count_nonzero(dphi>10)+2 # a bit of a hack, sorry - but this works


In [104]:
# searches all the 1000 points to mark the max points
def find_points(dFonF, n): 
    
    data1 = dFonF[0,:] # choosing cell 1
    data2 = dFonF[1,:] #cell 2
    data3 = dFonF[2,:] #cell 3
    
    df1 = pd.DataFrame(data1, columns = ['data1']) #converting our dataset into a data frame
    df2 = pd.DataFrame(data2, columns = ['data2'])
    df3 = pd.DataFrame(data3, columns = ['data3'])
    
    n = 1000  # number of points to be checked 
    # finding local peaks in the graph
    df1['max'] = df1.iloc[argrelextrema(df1.data.values, np.greater_equal,
                    order=n)[0]]['data1']
    df2['max'] = df2.iloc[argrelextrema(df2.data.values, np.greater_equal,
                    order=n)[0]]['data2']
    df2['max'] = df3.iloc[argrelextrema(df3.data.values, np.greater_equal,
                    order=n)[0]]['data3']
    
    return df1['max'], df2['max'], df3['max']

df1, df2, df3 = find_points(dFonF, 1000)
# plot results

plt.scatter(df1.index, df1['max'], c='r')
plt.plot(df1.index, df1['data'])
plt.title('Flourescent Time Series for Cell 1', fontsize = 20)
plt.xlabel('Time (s)', fontsize = 15)
plt.ylabel('Flourescent Activity', fontsize = 15)
plt.show()

# plt.scatter(df2.index, df2['max'], c='r')
# plt.plot(df2.index, df2['data'])
# plt.title('Flourescent Time Series for Cell 1', fontsize = 20)
# plt.xlabel('Time (s)', fontsize = 15)
# plt.ylabel('Flourescent Activity', fontsize = 15)
# plt.show()


# plt.scatter(df3.index, df3['max'], c='r')
# plt.plot(df3.index, df['data'])
# plt.title('Flourescent Time Series for Cell 1', fontsize = 20)
# plt.xlabel('Time (s)', fontsize = 15)
# plt.ylabel('Flourescent Activity', fontsize = 15)
# plt.show()

AttributeError: 'DataFrame' object has no attribute 'data'

In [94]:
def get_lapresponse(sp, phi_bin, nlaps, nphi):
    response = np.zeros((nlaps,nphi, nlaps, nphi))
    lap = 0
    phi0 = phi_bin[0]
    lapflag = True  # flag to ensure we only increment lap counter on first time in bin
    for i in range(0,L):
        if  sp[i]:
            response[lap][phi_bin[i]-1] += 1
        # and augment lap counter
        if (phi_bin[i]==phi0 and lapflag==True):
            lap += 1
            lapflag=False # put down flag until we are past this bin
            #print(lap)
        if phi_bin[i]>phi0:
            lapflag=True

    return r

# %% some basic processing you will need
# bin the spatial variable phi
nbins_phi = 20
edges = np.arange(0,360,360/nbins_phi)
phi_bin = np.digitize(phi,edges)

# then how many laps does the mouse do?
# use big jumps in position to mark clocking around circular path
dphi = np.diff(phi_bin,axis=0)
#plt.plot(t[:-1],dphi,'r-')
nlaps = np.count_nonzero(dphi>10)+2 # a bit of a hack, sorry - but this works




train2 = find_points(dFonF, 500)

def binarise_response(resp, threshold):
    return np.where(resp >= threshold, 1, 0)

# Discretising into binary colour map
cmap = ListedColormap(["None", "blue"])

# Plots of first three cells
# cells_to_plot = [0, 1, 2]
# for i in cells_to_plot:
#     train_cell = train2[i,:]
#     response, lap = get_lapresponse(sp = train_cell, phi_bin = phi_bin, nlaps = nlaps,  nphi=edges.size)
#     bin_resp = binarise_response(response, 1)

train_cell = train2[1]
response, lap = get_lapresponse(sp = train_cell, phi_bin = phi_bin, nlaps = nlaps,  nphi=edges.size)
bin_resp = binarise_response(response, 1)

# for i in cells_to_plot:
#     train_cell = train2[i,:]
#     print("train_cell: ", train_cell)
#     response, lap = get_lapresponse(sp=train_cell, phi_bin=phi_bin, nlaps=nlaps, nphi=edges.size)
#     print("response: ", response)
#     print("lap: ", lap)
#     bin_resp = binarise_response(response, 1)

IndexError: invalid index to scalar variable.